# Field-Mappable Tuple List Example

In [3]:
import fmtl
from fmtl.contrib import * #Holds all helper functions

Let's create some data: FMTL works on tuple lists.

In [4]:
data = [("a", 1, ["a","b"],[[["a",["a","b"]],"b"]]), ("b", 2, ["b","a"],[[["a","b",["b"]],"b"]])]
rows = ("letter","num","seq","nested")

## Basics

`FMTL(tuplelist,rows=None,checks=None)` rows and checks arguments are mandatory:
 - rows is for friendly field adressing
 - checks, if provided, is called at init on each tuple.

In [5]:
tl = fmtl.FMTL(data,rows)
print("num is field #"+str(tl.f2i("num")))

num is field #1


Fields can be mapped using `tl.set_mapping(field,map_func,unk=None)` using a dictionnary or a function.

In [6]:
tl.set_mapping("letter",{"a":"Ah","b":"Bi"})
print("==> " + str(list(tl)))
print()
tl.set_mapping("num",lambda x:x+10)
print("==> " + str(list(tl)))

==> [('Ah', 1, ['a', 'b'], [[['a', ['a', 'b']], 'b']]), ('Bi', 2, ['b', 'a'], [[['a', 'b', ['b']], 'b']])]

==> [('Ah', 11, ['a', 'b'], [[['a', ['a', 'b']], 'b']]), ('Bi', 12, ['b', 'a'], [[['a', 'b', ['b']], 'b']])]


Dictionnary based mappings are automatically applied to sequences (nested or not).

In [7]:
tl.set_mapping("seq",{"a":"Ah","b":"Bi"})
print("==> " + str(list(tl)))
print()
tl.set_mapping("nested",{"a":"Ah","b":"Bi"})
print("==> " + str(list(tl)))

==> [('Ah', 11, ['Ah', 'Bi'], [[['a', ['a', 'b']], 'b']]), ('Bi', 12, ['Bi', 'Ah'], [[['a', 'b', ['b']], 'b']])]

==> [('Ah', 11, ['Ah', 'Bi'], [[['Ah', ['Ah', 'Bi']], 'Bi']]), ('Bi', 12, ['Bi', 'Ah'], [[['Ah', 'Bi', ['Bi']], 'Bi']])]


Mapping is done as iteration time in the object's `__get_item__()` method.

In [8]:
print(str(tl.tuplelist[0])+" --maps to--> "+str(tl[0]))

('a', 1, ['a', 'b'], [[['a', ['a', 'b']], 'b']]) --maps to--> ('Ah', 11, ['Ah', 'Bi'], [[['Ah', ['Ah', 'Bi']], 'Bi']])


You still can prebuild it explicitely, it's just a list after all.

In [9]:
builded = list(tl) #if you don't have too much data, else use a generator
print(builded)

[('Ah', 11, ['Ah', 'Bi'], [[['Ah', ['Ah', 'Bi']], 'Bi']]), ('Bi', 12, ['Bi', 'Ah'], [[['Ah', 'Bi', ['Bi']], 'Bi']])]


## Useful Helpers (fmtl.contrib)

#### Counter Based
- get_field_dict(fmtl, field, offset=0, max_count=-1, key_iter=None, iter_func=None): Based on stdlib Counter, returns a dictionnary based on unique values from fields from offset to maxcount. 
All tuples or tuples[[key_iter]] are used. iter_func is used to iterate values in tuples if they are not atomic/hashable (hashable unit generator)

In [30]:
print(get_field_dict(tl,"letter"))                                 #classic
print(get_field_dict(tl,"letter",offset=2))                        #+2 at mapping
print(get_field_dict(tl,"letter",max_count=1))                     #keeps only 1 item
print(get_field_dict(tl,"letter",max_count=1,key_iter=[1]))        #keeps only 1 item (but only sees idx [1])
print(get_field_dict(tl,"seq",iter_func=lambda x: (y for y in x))) #iterate on each field seq.

{'Ah': 0, 'Bi': 1}
{'Ah': 2, 'Bi': 3}
{'Ah': 0}
{'Bi': 0}
{'Ah': 0, 'Bi': 1}


- get_stats(fmtl, field, key_iter=None, verbose=False): returns simple stats on a field.

In [28]:
get_stats(tl,"letter")

({'Ah': 1, 'Bi': 1}, {'Ah': 0.5, 'Bi': 0.5})

#### Selective iteration:
- `indexed_iter(fmtl,idxs)` returns an iterable which iterates `fmtl` tuples using indexs in `idx`

In [35]:
print("all tuples")
for x in tl:
    print(x)
   
print('tuples selected by index')
for x in indexed_iter(tl,[1,0,0,1,0,1]):
    print(x)

all tuples
('Ah', 11, ['Ah', 'Bi'], [[['Ah', ['Ah', 'Bi']], 'Bi']])
('Bi', 12, ['Bi', 'Ah'], [[['Ah', 'Bi', ['Bi']], 'Bi']])
tuples selected by index
('Bi', 12, ['Bi', 'Ah'], [[['Ah', 'Bi', ['Bi']], 'Bi']])
('Ah', 11, ['Ah', 'Bi'], [[['Ah', ['Ah', 'Bi']], 'Bi']])
('Ah', 11, ['Ah', 'Bi'], [[['Ah', ['Ah', 'Bi']], 'Bi']])
('Bi', 12, ['Bi', 'Ah'], [[['Ah', 'Bi', ['Bi']], 'Bi']])
('Ah', 11, ['Ah', 'Bi'], [[['Ah', ['Ah', 'Bi']], 'Bi']])
('Bi', 12, ['Bi', 'Ah'], [[['Ah', 'Bi', ['Bi']], 'Bi']])
